In [1]:
# import preprocess as prep
import SLR_model
import numpy as np
import tensorflow as tf
from tensorflow import keras
import time
import os
import json
import importlib
from matplotlib import pyplot as plt
import preprocess
# from preprocess import person

# model will output multiple(5) results/sec, how are we gonna handle it?

2024-10-14 22:59:24.262278: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-14 22:59:24.269355: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-14 22:59:24.271544: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-14 22:59:24.277374: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1728914365.084672 17973

In [2]:
 # reload model file
importlib.reload(SLR_model)
importlib.reload(preprocess)

<module 'preprocess' from '/home/shd/BomNae-SLR/preprocess.py'>

In [3]:
# Configurations
load_size = 256 # number of data to be loaded at once
epochs = 20
batch_size = 16
save_dir = "saves"
load_dir = "saves"
# load_path = "C:/Users/jerry/Desktop/hly/2024-2/cap/BN_SLR/BomNae-SLR/checkpoints.keras"


In [4]:
model = SLR_model.get_model()

In [5]:
# hand input shape: (batch, time, h, w, channels)
# pose input shape: (batch, time, channel, features)

In [6]:
# # 세이브된 모델 로드
# load_path = os.join(load_dir, "check_00-00-00.keras")
# model = SLR_model.load_model(load_path)

In [7]:
# # 안될때 (강제중지 + 초기화)
# tf.keras.backend.clear_session()

In [8]:
# l_raws=[]
# r_raws=[]
# p_raws=[]
# y_raws=[]


# save_suffix = time.strftime("_%d-%H-%M", time.localtime(time.time()))
# check_path = os.path.join(save_dir, "check"+save_suffix)
# hist_path = os.path.join(save_dir, "hist"+save_suffix+".json")

# model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
#     filepath=f'{check_path}.keras',
#     monitor='binary_accuracy',
#     mode='max',
#     save_freq='epoch',
#     save_best_only=True)



# l_raw, r_raw, p_raw, y_raw = preprocess.load_data(f"1/1")

# l_raws.append(l_raw)
# r_raws.append(r_raw)
# p_raws.append(p_raw)
# y_raws.append(y_raw)


# l_train, each = SLR_model.serialize(l_raws)
# r_train, each = SLR_model.serialize(r_raws)
# p_train, each = SLR_model.serialize(p_raws, stride=2)
# y_train = np.repeat(y_raws, each)
# y_train = SLR_model.num_arr2bin(y_train, 12)
# x_train = (l_train, r_train, p_train)

# # dataset = SLR_model.convert_to_dataset(x_train, y_train, batch_size)
# hist = model.fit(x_train, y_train, batch_size = 2, epochs=epochs, callbacks=[model_checkpoint_callback])

# with open(hist_path, 'w') as file:
#     json.dump(hist.history, file)


In [ ]:
 # reload model file
importlib.reload(SLR_model)
importlib.reload(preprocess)
end_file=preprocess.getoutputdir()

save_suffix = time.strftime("_%d-%H-%M", time.localtime(time.time()))
# check_path = os.path.join(save_dir, "check"+save_suffix)
# hist_path = os.path.join(save_dir, "hist"+save_suffix+".json")

checkTime="_14-16-16"
check_path = os.path.join(save_dir, "check"+checkTime)
hist_path = os.path.join(save_dir, "hist"+checkTime+".json")

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=f'{check_path}.keras',
    monitor='categorical_accuracy',
    mode='max',
    save_freq='epoch',
    save_best_only=True)

start_person=15
start_count=1
start_word=" "
for i in range(1,17):
    if start_person>i:
        continue
    l_raws=[]
    r_raws=[]
    p_raws=[]
    y_raws=[]
    loss_weights_raws=[]

    for k in range(1,11):
        if start_count>k:
            continue
        elif start_count==k:
            start_count=0
        for j in sorted(os.listdir(os.path.join(preprocess.getoutputdir(),str(i)))):
            if len(l_raws)==0:
                start_word = j
            else:
                end_word = j
            l_raw, r_raw, p_raw, y_raw, loss_weights_raw = preprocess.load_data(f"{i}/{j}")
            l_raws.append(l_raw)
            r_raws.append(r_raw)
            p_raws.append(p_raw)
            y_raws.append(y_raw)
            loss_weights_raws.append(loss_weights_raw)
    
            if len(l_raws)>=load_size:
                with open('training_log.txt', 'a') as logs:
                    logs.write(f'{ time.strftime("%H-%M-%S", time.localtime(time.time()))}:{k}) person:{i} : {start_word} ~ {end_word}\n')  # 한 줄 쓰기
                print(f"{k}) person:{i} : {start_word} ~ {end_word}\n\n\n\n")
                l_train, each = SLR_model.serialize(l_raws)
                r_train, each = SLR_model.serialize(r_raws)
                p_train, each, sample_weights = SLR_model.serialize(p_raws, stride=2, loss_weights_list=loss_weights_raws)
                x_train = (l_train, r_train, p_train)
                
                y_train = np.repeat(y_raws, each)
                y_train = SLR_model.encode_onehot2d(y_train)
    
                dataset = SLR_model.convert_to_dataset(x_train, y_train, batch_size, sample_weights)
                hist = model.fit(dataset, epochs=epochs, callbacks=[model_checkpoint_callback])
                
                with open(hist_path, 'w') as file:
                    json.dump(hist.history, file)
                l_raws.clear()
                r_raws.clear()
                p_raws.clear()
                y_raws.clear()
                loss_weights_raws.clear()
        if len(l_raws)>0:
            l_train, each = SLR_model.serialize(l_raws)
            r_train, each = SLR_model.serialize(r_raws)
            p_train, each, sample_weights = SLR_model.serialize(p_raws, stride=2, loss_weights_list=loss_weights_raws)
            x_train = (l_train, r_train, p_train)
            
            y_train = np.repeat(y_raws, each)
            y_train = SLR_model.encode_onehot2d(y_train)
    
            dataset = SLR_model.convert_to_dataset(x_train, y_train, batch_size, sample_weights)
            hist = model.fit(dataset, epochs=epochs, callbacks=[model_checkpoint_callback])
            
            with open(hist_path, 'w') as file:
                json.dump(hist.history, file)

    

0001.npz
0002.npz
0003.npz
0004.npz
0005.npz
0006.npz
0007.npz
0008.npz
0009.npz
0010.npz
0011.npz
0012.npz
0013.npz
0014.npz
0015.npz
0016.npz
0017.npz
0018.npz
0019.npz
0020.npz
0021.npz
0022.npz
0023.npz
0024.npz
0025.npz
0026.npz
0027.npz
0028.npz
0029.npz
0030.npz
0031.npz
0032.npz
0033.npz
0034.npz
0035.npz
0036.npz
0037.npz
0038.npz
0039.npz
0040.npz
0041.npz
0042.npz
0043.npz
0044.npz
0045.npz
0046.npz
0047.npz
0048.npz
0049.npz
0050.npz
0051.npz
0052.npz
0053.npz
0054.npz
0055.npz
0056.npz
0057.npz
0058.npz
0059.npz
0060.npz
0061.npz
0062.npz
0063.npz
0064.npz
0065.npz
0066.npz
0067.npz
0068.npz
0069.npz
0070.npz
0071.npz
0072.npz
0073.npz
0074.npz
0075.npz
0076.npz
0077.npz
0078.npz
0079.npz
0080.npz
0081.npz
0082.npz
0083.npz
0084.npz
0085.npz
0086.npz
0087.npz
0088.npz
0089.npz
0090.npz
0091.npz
0092.npz
0093.npz
0094.npz
0095.npz
0096.npz
0097.npz
0098.npz
0099.npz
0100.npz
0101.npz
0102.npz
0103.npz
0104.npz
0105.npz
0106.npz
0107.npz
0108.npz
0109.npz
0110.npz
0111.npz
0

In [ ]:
plt.plot(range(len(hist.history['loss'])), hist.history['loss'])
plt.scatter(range(len(hist.history['loss'])), hist.history['loss'])
plt.xlabel("epochs")
plt.ylabel("loss")
plt.legend(["loss"])
plt.ylim((0,1))
plt.show()

In [11]:
print(os.listdir(os.path.join(preprocess.getoutputdir(),str(15))))

['1962.npz', '2811.npz', '0220.npz', '2020.npz', '1494.npz', '0966.npz', '0039.npz', '0170.npz', '2124.npz', '1381.npz', '2545.npz', '1198.npz', '2965.npz', '0860.npz', '2651.npz', '0064.npz', '2842.npz', '1834.npz', '0796.npz', '1738.npz', '0242.npz', '0295.npz', '2218.npz', '0041.npz', '1023.npz', '0310.npz', '1108.npz', '2169.npz', '0068.npz', '2802.npz', '2268.npz', '0694.npz', '1440.npz', '0326.npz', '0570.npz', '1350.npz', '1863.npz', '1143.npz', '2425.npz', '1983.npz', '1332.npz', '1954.npz', '1623.npz', '2371.npz', '1838.npz', '2966.npz', '1484.npz', '0586.npz', '2557.npz', '2761.npz', '1532.npz', '0615.npz', '1295.npz', '0212.npz', '0070.npz', '0019.npz', '1998.npz', '1202.npz', '1763.npz', '1570.npz', '2672.npz', '0574.npz', '2800.npz', '2388.npz', '0739.npz', '2718.npz', '0139.npz', '1876.npz', '1636.npz', '2633.npz', '1756.npz', '2725.npz', '1323.npz', '1236.npz', '1614.npz', '0245.npz', '0949.npz', '0105.npz', '2980.npz', '1569.npz', '0887.npz', '2541.npz', '2497.npz', '00

In [ ]:
model.metrics_names